In [21]:
import pandas as pd
import os
import plotly.express as px
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [22]:
file_path = r"C:\Users\gauld\OneDrive\Documents\Git\Energy-Forecasting\csvs\matlab_temp.xlsx"
csv_directory = r"C:\Users\gauld\OneDrive\Documents\Git\Energy-Forecasting\csvs"
target = 'SYSLoad'
future = 0
epd = 48
set_name = "matlab"

In [23]:
def plotseasonal(res, axes):
    res.observed[-200:].plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observed')
    res.trend[-200:].plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Trend')
    res.seasonal[-200:].plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    res.resid[-200:].plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Residual')

In [29]:
data = pd.read_excel(file_path).set_index("Date")

data['PrevDaySameHour'] = data[target].copy().shift(epd)
data['PrevWeekSameHour'] = data[target].copy().shift(epd*7)
data['Prev24HourAveLoad'] = data[target].copy().rolling(window=epd*7, min_periods=1).mean()
data['Weekday'] = data.index.dayofweek
data.loc[(data['Weekday'] < 5) & (data['Holiday'] == 0), 'IsWorkingDay'] = 1
data.loc[(data['Weekday'] > 4) | (data['Holiday'] == 1), 'IsWorkingDay'] = 0

dec_daily = seasonal_decompose(data[target], model='additive', period=epd)
dec_weekly = seasonal_decompose(data[target], model='additive', period=epd*7)

data['IntraDayTrend'] = dec_daily.trend
data['IntraDaySeasonal'] = dec_daily.seasonal
data['IntraWeekTrend'] = dec_weekly.trend
data['IntraWeekSeasonal'] = dec_weekly.seasonal

# develop an imputer to replace values that don't exist somehow???

data = data.dropna(how='any', axis='rows')
y = data[target].shift(-epd*future).reset_index(drop=True)
y = y.dropna(how='any', axis='rows')

future_dates = pd.Series(data.index[future*epd:])
outputs = pd.DataFrame({"Date": future_dates, "{0}".format(target): y})

# future > 10 needs addressing - it is not yet implemented
if future > 10:
    data = data[['DryBulb', 'DewPnt', 'Prev5DayHighAve', 'Prev5DayLowAve', 'Hour', 'Weekday', 'IsWorkingDay']]
else:
    data = data.drop("{0}".format(target), axis=1)

data_name = csv_directory + "/" + set_name + "_data_" + str(future) + ".csv"
output_name = csv_directory + "/" + set_name + "_outputs_" + str(future) + ".csv"

data.to_csv(data_name)
outputs.to_csv(output_name, index=False)

In [25]:
dec_daily = seasonal_decompose(data[target], model='additive', period=epd)
dec_weekly = seasonal_decompose(data[target], model='additive', period=epd*7)

data['IntraDayTrend'] = dec_daily.trend
data['IntraDaySeasonal'] = dec_daily.seasonal
data['IntraWeekTrend'] = dec_weekly.trend
data['IntraWeekSeasonal'] = dec_weekly.seasonal

In [26]:
# develop an imputer to replace values that don't exist somehow???

data = data.dropna(how='any', axis='rows')
y = data[target].shift(-epd*future).reset_index(drop=True)
y = y.dropna(how='any', axis='rows')

future_dates = pd.Series(data.index[future*epd:])
outputs = pd.DataFrame({"Date": future_dates, "{0}".format(target): y})

In [28]:
# future > 10 needs addressing - it is not yet implemented
if future > 10:
    data = data[['DryBulb', 'DewPnt', 'Prev5DayHighAve', 'Prev5DayLowAve', 'Hour', 'Weekday', 'IsWorkingDay']]
else:
    data = data.drop("{0}".format(target), axis=1)

data_name = csv_directory + "/" + set_name + "_data_" + str(future) + ".csv"
output_name = csv_directory + "/" + set_name + "_outputs_" + str(future) + ".csv"

data.to_csv(data_name)
outputs.to_csv(output_name, index=False)